In [1]:
def get_xgb_stack_data(params,rounds,train,col,label):
    train = train.reset_index(drop=True)
    kf = KFold(n_splits=5,shuffle=False)
    i=0
    for train_index, test_index in kf.split(train):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
        y_train, y_test = label[train_index],label[test_index]
        dtrain = xgb.DMatrix(X_train[col],y_train)
        dtest = xgb.DMatrix(X_test[col])
        model = xgb.train(params,dtrain,num_boost_round=rounds)
        pred = model.predict(dtest)
        X_test['label'] = y_test
        X_test['predicted'] = pred
        if i==0:
            Final = X_test
        else:
            Final = Final.append(X_test,ignore_index=True)
        i+=1
    return Final

In [2]:
def get_sklearn_stack_data(model,train,col,label):
    train = train.reset_index(drop=True)
    kf = KFold(n_splits=5,shuffle=False)
    i=0
    for train_index, test_index in kf.split(train):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
        y_train, y_test = label[train_index],label[test_index]
        model.fit(X_train[col],y_train)
        pred = model.predict(X_test)
        X_test['label'] = y_test
        X_test['predicted'] = pred
        if i==0:
            Final = X_test
        else:
            Final = Final.append(X_test,ignore_index=True)
        i+=1
    return Final

In [3]:
def get_additional_features(train,test,macro):
    # Add month-year
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
    month_year_cnt_map = month_year.value_counts().to_dict()
    train['month_year_cnt'] = month_year.map(month_year_cnt_map)
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    month_year = (test.timestamp.dt.month + test.timestamp.dt.year * 100)
    month_year_cnt_map = month_year.value_counts().to_dict()
    test['month_year_cnt'] = month_year.map(month_year_cnt_map)
    # Add week-year count
    week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
    week_year_cnt_map = week_year.value_counts().to_dict()
    train['week_year_cnt'] = week_year.map(week_year_cnt_map)
    week_year = (test.timestamp.dt.weekofyear + test.timestamp.dt.year * 100)
    week_year_cnt_map = week_year.value_counts().to_dict()
    test['week_year_cnt'] = week_year.map(week_year_cnt_map)
    # Add month and day-of-week
    train['month'] = train.timestamp.dt.month
    train['dow'] = train.timestamp.dt.dayofweek
    test['month'] = test.timestamp.dt.month
    test['dow'] = test.timestamp.dt.dayofweek
    # Other feature engineering
    train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
    train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)
    test['rel_floor'] = test['floor'] / test['max_floor'].astype(float)
    test['rel_kitch_sq'] = test['kitch_sq'] / test['full_sq'].astype(float)
    train.apartment_name=train.sub_area.astype(str) + train['metro_km_avto'].astype(str)
    test.apartment_name=test.sub_area.astype(str) + train['metro_km_avto'].astype(str)
    train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
    test['room_size'] = test['life_sq'] / test['num_room'].astype(float)
    macro_cols = ['timestamp',"balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract",
    "micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate",
    "income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build"]
    macro = macro[macro_cols]
    train = train.merge(macro,how='left',on='timestamp')
    test = test.merge(macro,how='left',on='timestamp')
    def get_ratio_preschool(df):
        df['ratio_preschool'] = df['children_preschool'] / (df['children_school'] + 1)
        return df

    train = get_ratio_preschool(train)
    test = get_ratio_preschool(test)

    def get_extra_area(df):
        df['extra_area'] = df['full_sq'] - df['life_sq']
        return df

    train = get_extra_area(train)
    test = get_extra_area(test)

    def get_floor_ratio(df):
        df['floor_ratio'] = df['max_floor'] - df['floor']
        return df

    train = get_floor_ratio(train)
    test = get_floor_ratio(test)

    def get_room_avg_size(df):
        df['room_avg_size'] = (df['life_sq'] - df['kitch_sq']) / (df['num_room'] + 1)
        return df

    train = get_room_avg_size(train)
    test = get_room_avg_size(test)
    return train,test